In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers
import matplotlib.pyplot as plt
from sklearn import *
import math 
import random
import itertools
from sklearn.feature_selection import mutual_info_regression
import statistics 


In [2]:
# data
d=5
L=2000
e=2.71828


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1.0 / tf.sqrt(in_dim / 2.0)
    return tf.random_normal(shape=size, stddev=xavier_stddev)



def log2(x):
    numerator = tf.log(x)
    denominator = tf.log(tf.constant(2, dtype=numerator.dtype),)
    return numerator / denominator

def func(x):
    return x


def gen_X():
    return np.random.multivariate_normal( mean=meu,
                                  cov=eta_X,
                                  size = L)


def gen_Y(X):
    return func(X)+ np.random.multivariate_normal( mean=meu, cov=eta_N, size = L)



In [3]:
def ma(array, policy,window_size=None, beta=None):
    x=np.zeros(len(array))
    if policy=='window':
        for i in range(0, len(array)):
            if i<window_size:
                x[i]= np.mean(array[0: i])
            else:
                x[i]= np.mean(array[i-window_size: i])
    elif policy=='weighted':
        x[0]=array[0]
        for i in range(1, len(array)):
               x[i]=x[i-1]*(1-beta)+beta*array[i]
    elif policy=='None':
        for i in range(1, len(array)):
             x[i]=array[i] 
    return x


In [4]:
def MINE_5(x, y):


    L1=20
    L2=10
    L3=5
    L4=5
    L5=5

    x_shuffle=tf.random_shuffle(x)
    y_shuffle=tf.random_shuffle(y)
    In = tf.concat([x, y], axis=1)
    In_shuffle = tf.concat([x, y_shuffle], axis=1)

   # x=tf.reshape(x,[-1,1])
  #  y=tf.reshape(y,[-1,1])
   # x_shuffle=tf.reshape(x_shuffle,[-1,1])
   # y_shuffle=tf.reshape(y_shuffle,[-1,1])


    W1=tf.Variable(xavier_init([2*d, L1]))
    b1=tf.Variable(tf.zeros([L1]))

    layer_joint1=tf.nn.relu(tf.matmul(In,W1)+b1)
    layer_marg1=tf.nn.relu(tf.matmul(In_shuffle,W1)+b1)
    
    Wh2=tf.Variable(xavier_init( [L1,L2]))
    bh2=tf.Variable(tf.zeros([L2]))

    layer_joint2=tf.nn.relu(tf.matmul(layer_joint1,Wh2)+bh2)
    layer_marg2=tf.nn.relu(tf.matmul(layer_marg1,Wh2)+bh2)


    Wh3=tf.Variable(xavier_init( [L2,L3]))
    bh3=tf.Variable(tf.zeros([L3]))

    layer_joint3=tf.nn.relu(tf.matmul(layer_joint2,Wh3)+bh3)
    layer_marg3=tf.nn.relu(tf.matmul(layer_marg2,Wh3)+bh3)

    Wh4=tf.Variable(xavier_init( [L3,L4]))
    bh4=tf.Variable(tf.zeros([L4]))

    layer_joint4=tf.nn.relu(tf.matmul(layer_joint3,Wh4)+bh4)
    layer_marg4=tf.nn.relu(tf.matmul(layer_marg3,Wh4)+bh4)
    
    
    Wout=tf.Variable(xavier_init( [L4,1]))
    bout=tf.Variable(tf.zeros([1]))

    out_joint=tf.matmul(layer_joint4,Wout)+bout
    out_marg=tf.matmul(layer_marg4,Wout)+bout

    lower_bound=(tf.reduce_mean(out_joint,axis=0)-log2(tf.reduce_mean(tf.math.pow(2.0,out_marg),axis=0)+10e-5))

    theta = [W1,b1,Wh2, bh2,Wh3,bh3, Wh4,bh4 ,Wout, bout]

    opt = tf.train.AdamOptimizer(0.01).minimize((-lower_bound), var_list=[theta])

    return lower_bound, opt

In [5]:
def MINE_6(x, y):


    L1=20
    L2=10
    L3=5
    L4=5
    L5=5

    x_shuffle=tf.random_shuffle(x)
    y_shuffle=tf.random_shuffle(y)
    In = tf.concat([x, y], axis=1)
    In_shuffle = tf.concat([x, y_shuffle], axis=1)

   # x=tf.reshape(x,[-1,1])
  #  y=tf.reshape(y,[-1,1])
   # x_shuffle=tf.reshape(x_shuffle,[-1,1])
   # y_shuffle=tf.reshape(y_shuffle,[-1,1])


    W1=tf.Variable(xavier_init([2*d, L1]))
    b1=tf.Variable(tf.zeros([L1]))

    layer_joint1=tf.nn.relu(tf.matmul(In,W1)+b1)
    layer_marg1=tf.nn.relu(tf.matmul(In_shuffle,W1)+b1)
    
    Wh2=tf.Variable(xavier_init( [L1,L2]))
    bh2=tf.Variable(tf.zeros([L2]))

    layer_joint2=tf.nn.relu(tf.matmul(layer_joint1,Wh2)+bh2)
    layer_marg2=tf.nn.relu(tf.matmul(layer_marg1,Wh2)+bh2)


    Wh3=tf.Variable(xavier_init( [L2,L3]))
    bh3=tf.Variable(tf.zeros([L3]))

    layer_joint3=tf.nn.relu(tf.matmul(layer_joint2,Wh3)+bh3)
    layer_marg3=tf.nn.relu(tf.matmul(layer_marg2,Wh3)+bh3)

    Wh4=tf.Variable(xavier_init( [L3,L4]))
    bh4=tf.Variable(tf.zeros([L4]))

    layer_joint4=tf.nn.relu(tf.matmul(layer_joint3,Wh4)+bh4)
    layer_marg4=tf.nn.relu(tf.matmul(layer_marg3,Wh4)+bh4)
    
    Wh5=tf.Variable(xavier_init( [L4,L5]))
    bh5=tf.Variable(tf.zeros([L5]))

    layer_joint5=tf.nn.relu(tf.matmul(layer_joint4,Wh5)+bh5)
    layer_marg5=tf.nn.relu(tf.matmul(layer_marg4,Wh5)+bh5)
    
    Wout=tf.Variable(xavier_init( [L5,1]))
    bout=tf.Variable(tf.zeros([1]))

    out_joint=abs(tf.matmul(layer_joint5,Wout)+bout)
    out_marg=abs(tf.matmul(layer_marg5,Wout)+bout)

    lower_bound=tf.reduce_mean(out_joint,axis=0)-log2(tf.reduce_mean(tf.math.pow(2.0,out_marg),axis=0))

    theta = [W1,b1,Wh2, bh2,Wh3,bh3, Wh4,bh4 ,Wh5,bh5 ,Wout, bout]

    opt = tf.train.AdamOptimizer(0.01).minimize((-lower_bound), var_list=[theta])


    return lower_bound, opt

In [6]:
'''#alternative achitecture

L1=20
L2=10
L3=5
L4=5
L5=5
n_epochs = 2000
data_size = 20000
def MINE_alt(x_in, y_in):
    
    # shuffle and concatenate
    y_shuffle = tf.random_shuffle(y_in)
    x_conc = tf.concat([x_in, x_in], axis=0)
    y_conc = tf.concat([y_in, y_shuffle], axis=0)
    In=tf.concat([x_conc,y_conc], axis=1)
    # propagate the forward pass

    layer1 = tf.contrib.layers.fully_connected(In, L1,weights_initializer=layers.xavier_initializer(), activation_fn=tf.nn.tanh)
    layer2 = tf.contrib.layers.fully_connected(layer1, L2,weights_initializer=layers.xavier_initializer(), activation_fn=tf.nn.tanh)
    layer3 = tf.contrib.layers.fully_connected(layer2, L3,weights_initializer=layers.xavier_initializer(), activation_fn=tf.nn.tanh)
    layer4 = tf.contrib.layers.fully_connected(layer3, L4,weights_initializer=layers.xavier_initializer(), activation_fn=tf.nn.tanh)
    layer5 = tf.contrib.layers.fully_connected(layer4, L5,weights_initializer=layers.xavier_initializer(), activation_fn=tf.nn.tanh)
    output = tf.contrib.layers.fully_connected(layer5, 1,weights_initializer=layers.xavier_initializer(), activation_fn=None)

    
    # split in T_xy and T_x_y predictions
    N_samples = L
    T_xy = output[:N_samples]
    T_x_y = output[N_samples:]
    # compute the negative loss (maximise loss == minimise -loss)
    lower_bound = (tf.reduce_mean(T_xy, axis=0) - log2(tf.reduce_mean(tf.math.pow(2.0,T_x_y)))+10e-5)
    opt = tf.train.AdamOptimizer(learning_rate=0.01).minimize(-neg_loss)
    
    return lower_bound, opt
        '''

'#alternative achitecture\n\nL1=20\nL2=10\nL3=5\nL4=5\nL5=5\nn_epochs = 2000\ndata_size = 20000\ndef MINE_alt(x_in, y_in):\n    \n    # shuffle and concatenate\n    y_shuffle = tf.random_shuffle(y_in)\n    x_conc = tf.concat([x_in, x_in], axis=0)\n    y_conc = tf.concat([y_in, y_shuffle], axis=0)\n    In=tf.concat([x_conc,y_conc], axis=1)\n    # propagate the forward pass\n\n    layer1 = tf.contrib.layers.fully_connected(In, L1,weights_initializer=layers.xavier_initializer(), activation_fn=tf.nn.tanh)\n    layer2 = tf.contrib.layers.fully_connected(layer1, L2,weights_initializer=layers.xavier_initializer(), activation_fn=tf.nn.tanh)\n    layer3 = tf.contrib.layers.fully_connected(layer2, L3,weights_initializer=layers.xavier_initializer(), activation_fn=tf.nn.tanh)\n    layer4 = tf.contrib.layers.fully_connected(layer3, L4,weights_initializer=layers.xavier_initializer(), activation_fn=tf.nn.tanh)\n    layer5 = tf.contrib.layers.fully_connected(layer4, L5,weights_initializer=layers.xavie

In [7]:
n_epochs = 10000

# train
batch_size=5000
d=10
error_6=np.zeros(batch_size)
for i in range (batch_size):
    MIs = []
    meu=np.zeros(d)
    eta_X=datasets.make_spd_matrix(d, random_state=None)
    eta_N=datasets.make_spd_matrix(d, random_state=None)   
    eta_Y=eta_X+eta_N


    x=gen_X()
    y=gen_Y(x)
    mi = 0.5*np.log2(np.linalg.det(eta_Y)/np.linalg.det(eta_N))


    # prepare the placeholders for inputs
    x_in = tf.placeholder(tf.float32, [None, d], name='x_in')
    y_in = tf.placeholder(tf.float32, [None, d], name='y_in')
    # make the loss and optimisation graphs
    lower_bound, train_step= MINE_6(x_in, y_in)

    # start the session
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    # train
    MIs = []

    for epoch in range(n_epochs):

        # generate the data
        x_sample=gen_X()
        y_sample=gen_Y(x_sample)


        # perform the training step 
        feed_dict = {x_in:x_sample, y_in:y_sample}
        low_bnd,_ = sess.run([lower_bound, train_step], feed_dict=feed_dict)
        # print (te)
        # save the loss
        MIs.append(low_bnd)

        print(epoch/n_epochs*100,end='\r')
    print('\r')


    mv_av=ma(MIs, policy='weighted', beta=0.01)
    error_6[i]=((np.amax(mv_av)-mi)/mi)*100
    
        
    op = open("d={d}.txt".format(d=d),"a") 

    op.write( '{:} True value: {:.4} ; calculated value: {:.4}; error: {:.4}%; Average{:.4}% \n'
    .format(i+1, mi, np.amax(mv_av), error_6[i], np.mean(error_6[:i+1])) )
    op.close() 
    sess.close()
    print (i+1, '%')
error_6=error_6.tolist()
error_6.remove(max(error_6))
error_6.remove(min(error_6))
error_av_6=statistics.mean(error_6)



op = open("d={d}.txt".format(d=d),"a") 

op.write( 'Average error:{:.4} \n'
.format(error_av_6))
op.close()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
99.9900000000000143
1 %
99.9900000000000143
2 %
99.9900000000000143
3 %
99.9900000000000143
4 %
99.9900000000000143
5 %
99.9900000000000143
6 %
99.9900000000000143
7 %
99.9900000000000143
8 %
99.9900000000000143
9 %
99.9900000000000143
10 %
99.9900000000000143
11 %
99.9900000000000143
12 %
99.9900000000000143
13 %
99.9900000000000143
14 %
99.9900000000000143
15 %
99.9900000000000143
16 %
99.9900000000000143
17 %
99.9900000000000143
18 %
99.9900000000000143
19 %
99.9900000000000143
20 %
99.9900000000000143
21 %
99.9900000000000143
22 %
99.9900000000000143
23 %
99.9900000000000143
24 %
99.9900000000000143
25 %
99.9900000000000143
26 %
99.9900000000000143
27 %
99.9900000000000143
28 %
99.9900000000000143
29 %
99.9900000000000143
30 %
99.9900000000000143
31 %
99.9900000000000143
32 %
99.9900000000000143
33 %
99.9900000000000143
34 %
99.9900000000000143
35 %
99.99000000000

99.9900000000000143
296 %
99.9900000000000143
297 %
99.9900000000000143
298 %
99.9900000000000143
299 %
99.9900000000000143
300 %
99.9900000000000143
301 %
99.9900000000000143
302 %
99.9900000000000143
303 %
99.9900000000000143
304 %
99.9900000000000143
305 %
99.9900000000000143
306 %
99.9900000000000143
307 %
99.9900000000000143
308 %
99.9900000000000143
309 %
99.9900000000000143
310 %
99.9900000000000143
311 %
99.9900000000000143
312 %
99.9900000000000143
313 %
99.9900000000000143
314 %
99.9900000000000143
315 %
99.9900000000000143
316 %
99.9900000000000143
317 %
99.9900000000000143
318 %
99.9900000000000143
319 %
99.9900000000000143
320 %
99.9900000000000143
321 %
99.9900000000000143
322 %
99.9900000000000143
323 %
99.9900000000000143
324 %
99.9900000000000143
325 %
99.9900000000000143
326 %
99.9900000000000143
327 %
99.9900000000000143
328 %
99.9900000000000143
329 %
99.9900000000000143
330 %
99.9900000000000143
331 %
99.9900000000000143
332 %
99.9900000000000143
333 %
99.990000000

KeyboardInterrupt: 

In [9]:
import csv


err_file = "data.csv"

#Assuming res is a flat list
with open(err_file, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in error_6:
        writer.writerow([val])    